<H1>Building a Regression Model in Keras<H1>

# Table of Contents


<font size = 3>
    
1. Download and Clean Dataset
2. Import Keras  
3. Build a Neural Network  
4. Train and Test the Network  
5. Evaluating the model on the test data and computing the mean squared error between the predicted concrete strength and the actual concrete strength. 
6. Create a list of 50 mean squared errors 
7. Report the mean and the standard deviation of the mean squared errors. 

</font>
</div>

<h2>Download and clean dataset<h2>

In [3]:
import pandas as pd
import numpy as np

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

**_Now we will download the data and put it in pandas dataframe_**

In [10]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Let's check how many data points we have.

In [11]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

In [12]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


Now to check the data set for the missing values

In [13]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

So, our data have no missing value

**_Now to separate the prectictor columns and the target column_** 

In [15]:
concrete_columns= concrete_data.columns
predictor= concrete_data[concrete_columns[concrete_columns!= "Strength"]]
target = concrete_data["Strength"]

now lets check both the predictors and the target dataframes

In [16]:
predictor.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [17]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

_For the finding the no of predictor which is to be provided as an input to the regression model_

In [20]:
no_of_predictors= predictor.shape[1]
no_of_predictors

8

<h2>Importing Keras<h2>

In [24]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Let's import the rest of the packages from the Keras library that we will need to build our regression model.

In [26]:
from keras.models import Sequential
from keras.layers import Dense

<H1>Building the Neural Network for the regression model<H1>

In [39]:
def regression_model():
    model= Sequential()
    #designing with 1 hidden layer having 10 nodes and output layer having one node.
    model.add(Dense(10, activation= 'relu', input_shape=(no_of_predictors,)))
    model.add(Dense(1))
    
    #use the adam optimizer
    model.compile(optimizer= 'adam', loss= 'mean_squared_error')
    return model
    

**_1. At first importing the train_test_split and then split the data into training and test data_**

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, y_train, y_test = train_test_split(predictor, target, test_size=0.3, random_state=42)

In [35]:
X_train.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
196,194.7,0.0,100.5,165.6,7.5,1006.4,905.9,28
631,325.0,0.0,0.0,184.0,0.0,1063.0,783.0,7
81,318.8,212.5,0.0,155.7,14.3,852.1,880.4,3
526,359.0,19.0,141.0,154.0,10.9,942.0,801.0,3
830,162.0,190.0,148.0,179.0,19.0,838.0,741.0,28


In [36]:
y_train.head()

196    25.72
631    17.54
81     25.20
526    23.64
830    33.76
Name: Strength, dtype: float64

**<h2>Training and Testing the Network<h2>**

**_2. Taking the epochs to be 50_**

In [40]:
model= regression_model()

In [42]:
no_of_epochs= 50
model.fit(X_train, y_train, epochs= no_of_epochs, verbose=1) #verbose = 1, which includes both progress bar and one line per epoch.

Epoch 1/50
721/721 [==============================] - 1s 877us/step - loss: 22713.3404
Epoch 2/50
721/721 [==============================] - 0s 311us/step - loss: 5488.0769
Epoch 3/50
721/721 [==============================] - 0s 380us/step - loss: 2553.3258
Epoch 4/50
721/721 [==============================] - 0s 305us/step - loss: 2330.2060
Epoch 5/50
721/721 [==============================] - 0s 308us/step - loss: 2121.6774
Epoch 6/50
721/721 [==============================] - 0s 369us/step - loss: 1951.5726
Epoch 7/50
721/721 [==============================] - 0s 375us/step - loss: 1794.6158
Epoch 8/50
721/721 [==============================] - 0s 360us/step - loss: 1649.6700
Epoch 9/50
721/721 [==============================] - 0s 304us/step - loss: 1526.0119
Epoch 10/50
721/721 [==============================] - 0s 306us/step - loss: 1400.6482
Epoch 11/50
721/721 [==============================] - 0s 330us/step - loss: 1293.8168
Epoch 12/50
721/721 [==============================

**_3. Evaluating the model on the test data and computing the mean squared error between the predicted concrete strength and the actual concrete strength._**

In [45]:
loss= model.evaluate(X_test, y_test)
loss

309/309 [==============================] - 0s 83us/step


168.4960102957815

Computing the mean_squared error

In [47]:
from sklearn.metrics import mean_squared_error

In [48]:
y_pred=model.predict(X_test)

In [49]:
MSE = mean_squared_error(y_test, y_pred)
mean = np.mean(MSE)
standard_deviation = np.std(MSE)
print(mean, standard_deviation)

168.49601045061812 0.0


<h3>4. Create a list of 50 mean squared errors.<h3>

In [51]:
total_MSE = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_MSE):
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE no. {} IS {}:".format(i+1,MSE))
    y_pred = model.predict(X_test)
    MSE = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(MSE)

mean_squared_errors = np.array(mean_squared_errors)


MSE no. 1 IS 107.64928009131965:
MSE no. 2 IS 107.41310566528715:
MSE no. 3 IS 108.84395342583024:
MSE no. 4 IS 107.50133786000866:
MSE no. 5 IS 107.1611652806353:
MSE no. 6 IS 108.67304059371207:
MSE no. 7 IS 107.87237669812052:
MSE no. 8 IS 107.84249473003894:
MSE no. 9 IS 106.54481535976373:
MSE no. 10 IS 106.47209189702006:
MSE no. 11 IS 112.15347413492049:
MSE no. 12 IS 80.43820170677209:
MSE no. 13 IS 69.66966861119934:
MSE no. 14 IS 69.59244666979151:
MSE no. 15 IS 69.68445578516494:
MSE no. 16 IS 59.57718633756669:
MSE no. 17 IS 56.77390594235516:
MSE no. 18 IS 59.294156231926486:
MSE no. 19 IS 56.815815194139205:
MSE no. 20 IS 57.06542920603336:
MSE no. 21 IS 52.60725436164338:
MSE no. 22 IS 50.13974005196087:
MSE no. 23 IS 50.53862874793389:
MSE no. 24 IS 52.19505789056179:
MSE no. 25 IS 45.3005221345278:
MSE no. 26 IS 45.6525720762975:
MSE no. 27 IS 44.54810098074015:
MSE no. 28 IS 48.2599293915585:
MSE no. 29 IS 45.3910688813836:
MSE no. 30 IS 47.63749224468342:
MSE no. 31 

<h3>5. Report the mean and the standard deviation of the mean squared errors.<h3>

In [54]:
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("The mean of all the MSE is {}\n".format(mean))
print("Standard Deviation of all the MSE is {}: ".format(standard_deviation))



The mean of all the MSE is 64.4808496451875

Standard Deviation of all the MSE is 24.38497657551746: 


**<H1>THANK YOU<H1>**